In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import torch
import wandb

from config import Environment, TrainConfig
from denoising.train import prepare_training
from denoising.utils import seed_everything

In [3]:
CWD = Path.cwd()
env = Environment(_env_file=CWD / '../env')
wandb.login(key=env.wandb_api_key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/d.nesterov/.netrc
wandb: Currently logged in as: dmitrylala to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Конфиг обучения и подготовка

In [4]:
run_idx = 25
save_dir = CWD / 'mri'


cfg = TrainConfig(
    # Datasets params
    train_dset='mri_pm_train',
    test_dset='mri_pm_test',
    train_batch_size=64,
    test_batch_size=128,
    # Model params
    name_model='mri-fno-dense-2',
    cfg_fno={
        'n_modes': (32, 32),
        'in_channels': 1,
        'hidden_channels': 32,
        'lifting_channel_ratio': 8,
        'projection_channel_ratio': 2,
        'out_channels': 1,
        'factorization': 'tucker',
        'n_layers': 4,
        'rank': 0.42,
    },
    # Run params
    random_seed=42,
    device='cuda:2',
    run_name=f'Run {run_idx}, FNO dense',
    save_weights_path=save_dir / f'run-{run_idx}-weights.pt',
    # Train params
    n_epochs=50,
    lr=1e-3,
    wandb_log=True,
    save_dir=save_dir / f'run-{run_idx}',
    verbose=True,
)

In [5]:
trainer, train_kwargs, run = prepare_training(env, cfg)

Got n_samples = 8380  in dataset mri_pm_train        with sample size = torch.Size([1, 145, 145])
Got n_samples = 2093  in dataset mri_pm_test         with sample size = torch.Size([1, 145, 145])
Got n_samples = 2093  in dataset mri_gt_test         with sample size = torch.Size([1, 145, 145])
Got n_samples = 137   in dataset bsd_synth_0.01_train with sample size = torch.Size([1, 321, 481])
Got n_samples = 77    in dataset bsd_synth_0.01_test with sample size = torch.Size([1, 321, 481])
Got n_samples = 12296 in dataset sidd_train          with sample size = torch.Size([3, 512, 512])
Got n_samples = 3008  in dataset sidd_test           with sample size = torch.Size([3, 512, 512])
torch.Size([64, 1, 145, 145]) torch.Size([64, 1, 145, 145])
Loaded  model mri-fno-neuralop with n_parameters = 2010449
Loaded  model mri-fno-custom   with n_parameters = 2010449
Loaded  model mri-fno-dense    with n_parameters = 4476513
Loaded  model mri-hno-separable with n_parameters = 2248289
Loaded  model mr

Logging to wandb enabled: True


# Запуск

In [ ]:
seed_everything(cfg.random_seed)
trainer.train(**train_kwargs)

Training on 8380 samples
Testing on [2093] samples         on resolutions ['test'].
Raw outputs of shape torch.Size([64, 1, 145, 145])
[0] time=32.46, avg_loss=0.1919, train_err=12.2731
Eval: test_h1=0.1125, test_l2=0.0483
Saved training state to /home/d.nesterov/denoising-fno/notebooks/mri/run-25
[1] time=29.86, avg_loss=0.1083, train_err=6.9283
Eval: test_h1=0.1052, test_l2=0.0842
Saved training state to /home/d.nesterov/denoising-fno/notebooks/mri/run-25
[2] time=29.76, avg_loss=0.1011, train_err=6.4647
Eval: test_h1=0.0998, test_l2=0.0498
Saved training state to /home/d.nesterov/denoising-fno/notebooks/mri/run-25
[3] time=29.11, avg_loss=0.0971, train_err=6.2133
Eval: test_h1=0.0964, test_l2=0.0365
Saved training state to /home/d.nesterov/denoising-fno/notebooks/mri/run-25
[4] time=29.24, avg_loss=0.0914, train_err=5.8496
Eval: test_h1=0.0885, test_l2=0.0329
Saved training state to /home/d.nesterov/denoising-fno/notebooks/mri/run-25
[5] time=28.68, avg_loss=0.0862, train_err=5.5160

In [ ]:
if run is not None:
    run.finish()

In [ ]:
torch.save(trainer.model.to('cpu').state_dict(), cfg.save_weights_path)
print(f'Saved to {cfg.save_weights_path}')